Esse notebook será utilizado para comparar as bases finais geradas pelo ETL e pelo ELT.

In [ ]:
# Vamos fazer os imports necessários

import pandas as pd
import sqlite3
import numpy as np
import os

In [ ]:
# Conectar ao Data Warehouse criado pelo ELT
db_path = 'samu_dw.db'
if not os.path.exists(db_path):
    print("❌ ERRO: Arquivo samu_dw.db não encontrado. Rode o notebook ELT_SAMU.ipynb primeiro.")